<a href="https://colab.research.google.com/github/bascoul/TP_Deep_Learning/blob/master/word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Incrustation de mots (Word embeddings)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/word_embeddings">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/word_embeddings.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Ce tutoriel présente l'intégration de mots. Il contient le code complet pour former les incorporations de mots à partir de zéro sur un petit ensemble de données, et pour visualiser ces incorporations en utilisant le [Projecteur d'incorporation] (http://projector.tensorflow.org) (montré dans l'image ci-dessous).

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=1" alt="Capture d'écran du projecteur d'incorporation" width="400"/>

## Représentation du texte sous forme de chiffres

Les modèles d'apprentissage machine prennent des vecteurs (tableaux de nombres) comme entrée. Lorsque l'on travaille avec du texte, il faut d'abord trouver une stratégie pour convertir les chaînes de caractères en nombres (ou pour "vectoriser" le texte) avant de l'introduire dans le modèle. Dans cette section, nous allons examiner trois stratégies pour y parvenir.

### Codage à chaud

Comme première idée, nous pourrions encoder chaque mot de notre vocabulaire en une seule fois. Considérons la phrase "The cat sat on the mat". Le vocabulaire (ou les mots uniques) de cette phrase est (cat, mat, on, sat, the). Pour représenter chaque mot, nous allons créer un vecteur zéro de longueur égale au vocabulaire, puis placer un un dans l'index qui correspond au mot. Cette approche est illustrée dans le schéma suivant.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/one-hot.png?raw=1" alt="Diagramme des codages à chaud" width="400" />

Pour créer un vecteur qui contient l'encodage de la phrase, nous pourrions ensuite concaténer les vecteurs d'un seul mot.

Point clé : Cette approche est inefficace. Un vecteur codé en une seule fois est rare (c'est-à-dire que la plupart des indices sont nuls). Imaginez que nous ayons 10 000 mots dans le vocabulaire. Pour coder chaque mot en une seule fois, nous créerions un vecteur où 99,99% des éléments sont nuls.

### Encodez chaque mot avec un nombre unique

Une deuxième approche que nous pourrions essayer est de coder chaque mot en utilisant un numéro unique. En poursuivant l'exemple ci-dessus, nous pourrions attribuer 1 à "cat", 2 à "mat", et ainsi de suite. Nous pourrions ensuite coder la phrase "The cat sat on the mat" comme un vecteur dense comme [5, 1, 4, 3, 5, 2]. Cette application est efficace. Au lieu d'un vecteur épars, nous avons maintenant un vecteur dense (où tous les éléments sont pleins).

Il y a cependant deux inconvénients à cette approche :

* Le codage des entiers est arbitraire (il ne capture aucune relation entre les mots).

* Un codage d'entier peut être difficile à interpréter pour un modèle. Un classificateur linéaire, par exemple, apprend un seul poids pour chaque caractéristique. Comme il n'y a pas de relation entre la similarité de deux mots et la similarité de leurs codages, cette combinaison caractéristique - poids n'est pas significative.

### Encodage des mots

Les incrustations de mots nous permettent d'utiliser une représentation efficace et dense dans laquelle des mots similaires ont un encodage similaire. Il est important de noter que nous n'avons pas besoin de spécifier ce codage à la main. Une incrustation est un vecteur dense de valeurs en virgule flottante (la longueur du vecteur est un paramètre que vous spécifiez). Au lieu de spécifier les valeurs de l'incrustation manuellement, ce sont des paramètres entraînables (poids appris par le modèle pendant l'entraînement, de la même manière qu'un modèle apprend les poids pour une couche dense). Il est courant de voir des incrustations de mots en 8 dimensions (pour de petits ensembles de données), jusqu'à 1024 dimensions lorsque l'on travaille avec de grands ensembles de données. Une incrustation à plus haute dimension peut saisir des relations fines entre les mots, mais nécessite plus de données pour l'apprentissage.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding2.png?raw=1" alt="Diagramme d'un encastrement" width="400"/>

Ci-dessus se trouve un diagramme pour l'incrustation d'un mot. Chaque mot est représenté sous la forme d'un vecteur à 4 dimensions de valeurs en virgule flottante. Une autre façon d'envisager l'intégration est d'utiliser une "table de consultation". Après avoir appris ces poids, nous pouvons coder chaque mot en recherchant le vecteur dense auquel il correspond dans le tableau.

## Setup

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf

In [0]:
from tensorflow import keras
from tensorflow.keras import layers
# On utilise le module tfds pour charger la base de données : https://www.tensorflow.org/datasets/api_docs/python/tfds
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## Utilisation de la couche d'incrustation

Keras facilite l'utilisation de l'incrustation des mots. Jetons un coup d'œil à la couche [Embedding] (https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding).

La couche Embedding peut être comprise comme une table de recherche qui permet de faire la correspondance entre les indices entiers (qui représentent des mots spécifiques) et les vecteurs denses (leurs incrustations). La dimensionnalité (ou largeur) de l'incrustation est un paramètre que vous pouvez expérimenter pour voir ce qui fonctionne bien pour votre problème, de la même manière que vous expérimenteriez avec le nombre de neurones dans une couche dense.



In [0]:
embedding_layer = layers.Embedding(1000, 5)



Lorsque vous créez une couche d'incrustation, les poids pour l'incrustation sont initialisés de manière aléatoire (comme pour toute autre couche). Au cours de l'apprentissage, ils sont progressivement ajustés par rétropropagation. Une fois l'apprentissage terminée, les encastrements de mots appris coderont approximativement les similarités entre les mots (comme ils ont été appris pour le problème spécifique sur lequel votre modèle est appris).

Si vous passez un entier à une couche d'incrustation, le résultat remplace chaque entier par le vecteur de la table d'uncrustation :

In [0]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

Pour les problèmes de texte ou de séquence, la couche d'incrustation prend un tenseur 2D d'entiers, de forme `(samples, sequence_length)`, où chaque entrée est une séquence d'entiers. Elle peut incorporer des séquences de longueurs variables. Vous pouvez alimenter la couche d'incrustation ci-dessus des lots de formes `(32, 10)` (lot de 32 séquences de longueur 10) ou `(64, 15)` (lot de 64 séquences de longueur 15).

Le tenseur retourné a un axe de plus que l'entrée, les vecteurs d'incustration sont alignés le long du nouvel axe. Passez-lui un lot d'entrée `(2, 3)` et la sortie sera `(2, 3, N)`.


In [0]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

Lorsqu'on lui donne un lot de séquences en entrée, une couche d'incrustation renvoie un tenseur 3D à virgule flottante, de forme `(samples, sequence_length, embedding_dimensionality)`. Pour convertir cette séquence de longueur variable en une représentation fixe, il existe diverses approches standard. Vous pouvez utiliser une couche RNN, d'Attention, ou de regroupement avant de la passer à une couche Dense. Ce tutoriel utilise le pooling parce que c'est le plus simple. Le tutoriel [Classification de texte avec un RNN](text_classification_rnn.ipynb) est une bonne étape suivante.

## Apprendre l'incrustation à partir de zéro

Dans ce tutoriel, vous formerez un classificateur de sentiments sur les critiques de films IMDB. Dans le processus, le modèle apprendra les incrustations à partir de zéro. Nous utiliserons un ensemble de données prétraitées.

Pour charger un ensemble de données textuelles à partir de zéro, voir le [tutoriel sur le chargement de texte](../load_data/text.ipynb).

In [0]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

Obtenez l'encodeur (`tfds.features.text.SubwordTextEncoder`), et jetez un coup d'oeil rapide au vocabulaire. 

Les "\_" dans le vocabulaire représentent des espaces. Notez comment le vocabulaire comprend des mots entiers (se terminant par "\_") et des mots partiels qu'il peut utiliser pour construire des mots plus grands :

In [0]:
encoder = info.features['text'].encoder
encoder.subwords[:20]

Les critiques de films peuvent être de différentes longueurs. Nous utiliserons la méthode `padded_batch` pour standardiser la longueur des critiques.

In [0]:
train_data

In [0]:
train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes=([None],[]))
test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes=([None],[]))

Remarque : à partir de **TensorFlow 2.2**, l'argument padded_shapes n'est plus nécessaire. Le comportement par défaut est de capitonner tous les axes au plus long dans le lot.

In [0]:
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

Tel qu'il est importé, le texte des critiques est codé par des entiers (chaque entier représente un mot ou une partie de mot spécifique dans le vocabulaire).

Notez les zéros de fin de lot, car le lot est ajouté à l'exemple le plus long.

In [0]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()

Créer un modèle simple

Nous utiliserons l'API séquentielle de Keras pour définir notre modèle. Dans ce cas, il s'agit d'un modèle de type "sac de mots continu".

*   Ensuite, la couche d'incrustation prend le vocabulaire codé en entier et recherche le vecteur d'incorporation pour chaque index de mots. Ces vecteurs sont appris au fur et à mesure que le modèle apprend. Les vecteurs ajoutent une dimension au tableau de sortie. Les dimensions résultantes sont : (lot, séquence, incrustation).
*   Ensuite, une couche GlobalAveragePooling1D renvoie un vecteur de sortie de longueur fixe pour chaque exemple en faisant la moyenne sur la dimension de la séquence. Cela permet au modèle de traiter des entrées de longueur variable, de la manière la plus simple possible.
*   Ce vecteur de sortie de longueur fixe est acheminé à travers une couche entièrement connectée (dense) comportant 16 unités cachées.
*   La dernière couche est connectée de manière dense avec un seul noeud de sortie. En utilisant la fonction d'activation sigmoïde, cette valeur est un flottant entre 0 et 1, représentant une probabilité (ou un niveau de confiance) que l'examen soit positif.

Attention : Ce modèle n'utilise pas de masquage, donc le rembourrage de zéro est utilisé comme une partie de l'entrée, donc la longueur du rembourrage peut affecter la sortie. Pour y remédier, voir le guide sur le masquage et le remplissage.

In [0]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

model.summary()

### Compiler et apprendre le modèle

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

Avec cette approche, notre modèle atteint une précision de validation d'environ 88% (notez que le modèle est suradapté (overfitting), la précision de la formation est nettement plus élevée).

In [0]:
import matplotlib.pyplot as plt

history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss=history_dict['loss']
val_loss=history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()

## Récupérez les incrustations apprises

Ensuite, récupérons les mots intégrés appris pendant la formation. Il s'agira d'une matrice de forme (vocab_size, embedding-dimension).

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

Nous allons maintenant écrire les poids sur le disque. Pour utiliser le [Embedding Projector] (http://projector.tensorflow.org), nous allons télécharger deux fichiers au format séparé par des tabulations : un fichier de vecteurs (contenant l'incrustation) et un fichier de métadonnées (contenant les mots).

In [0]:
import io

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

Si vous exécutez ce tutoriel dans [Colaboratory](https://colab.research.google.com), vous pouvez utiliser l'extrait suivant pour télécharger ces fichiers sur votre machine locale (ou utiliser le navigateur de fichiers).

In [0]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

## Visualiser les incrustations

Pour visualiser nos incrustations, nous les téléchargerons sur le projecteur d'incrustation.

Ouvrez le [Embedding Projector] (http://projector.tensorflow.org/) (il peut également fonctionner dans une instance locale de TensorBoard).

* Cliquez sur "Load data".

* Téléchargez les deux fichiers que nous avons créés ci-dessus : `vecs.tsv` et `meta.tsv`.

Les incrustations que vous avez formées s'afficheront alors. Vous pouvez rechercher des mots pour trouver leurs voisins les plus proches. Par exemple, essayez de chercher "beautiful". Vous verrez peut-être des voisins comme "wonderful". 

Note : vos résultats peuvent être un peu différents, selon la façon dont les poids ont été initialisés au hasard avant l'entraînement de la couche d'incrustation.

Note : à titre expérimental, vous pourrez peut-être produire des incrustations plus interprétables en utilisant un modèle plus simple. Essayez de supprimer la couche "Dense(16)", de rééduquer le modèle, et de visualiser à nouveau les incrustations.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=1" alt="Capture d'écran du projecteur d'encastrement" width="400"/>
